# Pruning Regression Trees

Baum-basierte Ansätze teilen den Eingabe-Raum $\mathbb{R}^p$ rekursiv in $p$-dimensionale Quader $Q_m$ auf. Für jeden Quader wird im Fall des quadratischen Fehlers als Fehlermaß der Mittelwert $\hat c_m$ verwendet. Beim Trainieren eines Regressionsbaums besteht die Schwierigkeit, ein geeignetes Abbruchkriterium zu finden. Mögliche Kriterien weitere Aufteilungen vorzunehmen sind: 
- die maximale Baumtiefe, 
- die Fehlerreduzierung einer Aufteilung, 
- die Anzahl an Trainingsbeispielen in einem Knoten. 

Die Wahl der maximalen Baumtiefe oder einen Schwellwert für die Fehlerreduzierung ist nicht einfach im Vorfeld festzulegen. Zudem kann es sein, dass ein Schwellwert für die Fehlerreduzierung bei einer Aufteilung nicht überschritten wird, jedoch eine darauffolgende Aufteilung zu einer größeren Reduzierung führen würde.

Ein alternativer Ansatz zu diesen Abbruchkriterien ist das „*Pruning*“. Bei diesem Vorgehen werden die Abbruchkriterien sehr konservativ angesetzt und der Baum in Nachgang gestutzt. 

## Pruning

Es gibt verschiedene Ansätze das Pruning durchzuführen. Wir wollen uns hier das weakest-link-Pruning anschauen. Hierbei wird die Anpassung des Baumes an die Trainingsdaten mit der Komplexität des Baumes sanktioniert. 

Hierfür definiert man das Cost-Complexity-Kriterium $C_{\alpha}$ für einen Baum $T$:

$$C_{\alpha}(T) = \sum^{|T|}_{m=1}\sum_{x_{i}\in Q_m} (y_i-\hat c_m)^2 + \alpha |T| $$

Die Idee ist nun für jedes $\alpha$ einen Teilbaum $T_\alpha \subset T$ zu finden, der $C_{\alpha}(T)$ minimiert. Das $\alpha$ bestimmt man mittels eines Validierungsdatensatzes. 

Beim weakest-link-pruning sucht man sich den Knoten, der den kleinsten Anstieg pro Knoten von $\sum^{|T|}_{m=1}\sum_{x_{i}\in Q_m} (y_i-\hat c_m)^2$ zur Folge hat und fährt dann rekursiv fort, bis der Baum nur aus einem Knoten besteht. 

Die Fragen die sich nun stellen sind: 
- Wie finden wir die geeigneten Bäume in dieser Folge?
- Zu welchem $\alpha$ gehören diese Bäume?

Zur Beantwortung diese Frage stellen wir uns die Frage, bei welchem $\alpha$ wird ein Teilbaum $\tilde{T}$ besser als $T$?

Dazu schauen wir uns an, bei welchem $\alpha$ das 'Prunen' in einem Knoten $t$ dazu führt, dass die Kosten $C_{\alpha}(\tilde{T})$ des Teilbaum $\tilde{T}$ kleinere  sind als die Kosten $C_{\alpha}(T)$ von $T$ sind. Dies entspricht der Grenze bei der die Kosten im Knoten $t$ den Kosten in dem Teilbaum T_{t} entsprechen:


$$ 
\begin{align}
    C(T_t) &= C(t) \\
    \sum^{|T_t|}_{m=1}\sum_{x_{i}\in Q_m} (y_i-\hat c_m)^2 + \alpha |T_t| &= \sum_{x_{i}\in Q_t} (y_i-\hat c_t)^2 + \alpha \\
    \alpha |T_t| - \alpha &= \sum_{x_{i}\in Q_t} (y_i-\hat c_t)^2 - \sum^{|T_t|}_{m=1}\sum_{x_{i}\in Q_m} (y_i-\hat c_m)^2 \\
    \alpha &= \frac{\sum_{x_{i}\in Q_t} (y_i-\hat c_t)^2 - \sum^{|T_t|}_{m=1}\sum_{x_{i}\in Q_m} (y_i-\hat c_m)^2}{|T_t| - 1} \\
    \alpha &= g(t) \\
\end{align}
$$

Wir fangen bei $\alpha_k = 0$ an, berechnen das nächst größere $\alpha_{k+1}$. Wir prunen an dem inneren Knoten mit $min_t g_k(t)$ um den Teilbaum $T_k+1$ zu erhalten. 

Um die finale Wahl aus der Menge der Teilbäume $\{T_1, ...,T_K\}$ zu treffen, wählen wir den Baum mit dem geringsten Fehler auf einem Validierungs-Datensatz.

**Aufgabe:**
- Implementiere das Pruning mit Hilfe von ```sklearn.tree._tree```
- Teile die Daten in ```train```, ```validate```, ```test```. 
- Vergleiche den gestutzten Baum mit ungestutzten Bäumen verschiedener Tiefe. 


Beispielcode für das rekursive Durchlaufen eines Baumes:

In [1]:
from sklearn.tree import _tree

In [2]:
from sklearn.tree._tree import TREE_LEAF

def prune(inner_tree, index, threshold):
    # wenn es 'children' besuche die 'children'
    if inner_tree.children_left[index] != TREE_LEAF:
        prune(inner_tree, inner_tree.children_left[index], threshold)
        prune(inner_tree, inner_tree.children_right[index], threshold)
    
    # wenn es keine 'children' gibt bin ich ein leaf knoten
    else:
        print(index)